In [2]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset
import transformers
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,TrainingArguments,Trainer

from src.utils.myutils import *
import yaml
from tqdm import tqdm
import logging
import json
import warnings
import random

logging.disable(logging.ERROR)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
BATCH_SIZE = 64

In [2]:
data = load_dataset('csv',data_files = CS_DATA_PATH)['train']
with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)

## Training

In [3]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [3]:
#same args as in original paper
training_args = TrainingArguments(
    output_dir = './',
    num_train_epochs=3,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=50,
    disable_tqdm = False,
    save_total_limit=2,
    weight_decay=0.1,
    learning_rate=5e-5)

In [6]:
model_scores = {}

for model_name in tqdm(config_data['models']):
    scores = []
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    token_full = preprocess_data(data,tokenizer,'sentence')

    print("Running 10-fold CV on model: ",model_name,"...")
    for train_index, val_index in skfold.split(token_full['input_ids'],token_full['label']):

        token_train = Dataset.from_dict(token_full[train_index])
        token_valid = Dataset.from_dict(token_full[val_index])

        model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
        model.to(device)
        trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
        trainer.train()

        #evaluation
        eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
        scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
    print("Done.")
    print(np.mean(scores))
    model_scores[model_name] = scores
    

  0%|          | 0/6 [00:00<?, ?it/s]

Running 5-fold CV on model:  UWB-AIR/Czert-B-base-cased ...


Step,Training Loss
50,0.538500
100,0.326200


Step,Training Loss
50,0.520300
100,0.340900


Step,Training Loss
50,0.523500
100,0.327400


...

Step,Training Loss
50,0.566600
100,0.389800


Step,Training Loss
50,0.545100
100,0.392500


Step,Training Loss
50,0.555800
100,0.369100


100%|██████████| 6/6 [51:40<00:00, 516.68s/it]

Done.
0.753730624380135


In [8]:
for model_name in config_data['models']:
    print(model_name, "score:", np.mean(model_scores[model_name]))

UWB-AIR/Czert-B-base-cased score: 0.7759894788824253
ufal/robeczech-base score: 0.7736152637058488
bert-base-multilingual-cased score: 0.7335294970781417
fav-kky/FERNET-C5 score: 0.7803352776299648
fav-kky/FERNET-News score: 0.5659761591194752
DeepPavlov/bert-base-bg-cs-pl-ru-cased score: 0.753730624380135
